In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table, Column, ForeignKey,Integer

In [2]:
# Create Database Connection
engine = create_engine('sqlite:///../raw_data/full_data/wine.sqlite')
Base = automap_base()

In [3]:
Base.prepare(engine, reflect=True)
Base.classes.keys()

['reviews']

In [4]:
reviews=Base.classes.reviews
# point=Base.classes.prpt
session=Session(engine)

In [5]:
first_row=session.query(reviews).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2334eed5be0>,
 'country': 'US',
 'description': 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'designation': None,
 'id': 1,
 'points': 87.0,
 'price': 14.0,
 'province': 'Oregon',
 'region_1': 'Willamette Valley',
 'region_2': 'Willamette Valley',
 'taster_name': 'Paul Gregutt',
 'taster_twitter_handle': '@paulgwine\xa0',
 'title': 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 'variety': 'Pinot Gris',
 'winery': 'Rainstorm'}

In [6]:
titles=session.query(reviews.variety).distinct().count()
titles

257

In [16]:
state=[]
variety=[]
points=[]
price=[]
title_cnt=[]

for row in session.query(reviews.province,reviews.variety,func.avg(reviews.points).label('Average_Point'),\
                         func.avg(reviews.price).label('Average_Price'),func.count(reviews.title).label('Title_cnt')).group_by(reviews.province,\
                        reviews.variety).order_by(reviews.province).all():
    state.append(row.province)
    variety.append(row.variety)
    title_cnt.append(row.Title_cnt)
    points.append(round(row.Average_Point,2))
    if (row.Average_Price != None):
        price.append(round(row.Average_Price))
    else:
        price.append(0)


In [17]:
state_prpt=pd.DataFrame({"State":state,
                    "Variety":variety,
                    "Avg_Points":points,
                    "Avg_Price":price,
                    "Title_Count":title_cnt}, columns=['State','Variety','Avg_Points','Avg_Price','Title_Count'])

state_prpt.head()

,State,Variety,Avg_Points,Avg_Price,Title_Count
0,America,Barbera,83.00,18,1
1,America,Bordeaux-style Red Blend,87.00,38,2
2,America,Cabernet Sauvignon,85.50,20,10
3,America,Champagne Blend,87.00,16,1
4,America,Chardonnay,84.88,16,8


In [18]:
engine = create_engine('sqlite:///../raw_data/full_data/wine.sqlite')
state_prpt.to_sql('prpt', engine, if_exists='replace')

In [43]:
df=pd.read_sql('prpt',engine)
df.head()
st='Ohio'

In [20]:
df[(df['State']==st)]

,index,State,Variety,Avg_Points,Avg_Price,Title_Count
450,450,Ohio,Cabernet Franc,83.00,15,1
451,451,Ohio,Cabernet Sauvignon,84.00,12,1
452,452,Ohio,Chardonnay,85.50,14,2
453,453,Ohio,Nebbiolo,80.00,15,1
454,454,Ohio,Riesling,84.43,14,7


In [21]:
df[(df['State']==st)].to_json(orient='records')
# to_json(orient='records')

'[{"index":450,"State":"Ohio","Variety":"Cabernet Franc","Avg_Points":83.0,"Avg_Price":15,"Title_Count":1},{"index":451,"State":"Ohio","Variety":"Cabernet Sauvignon","Avg_Points":84.0,"Avg_Price":12,"Title_Count":1},{"index":452,"State":"Ohio","Variety":"Chardonnay","Avg_Points":85.5,"Avg_Price":14,"Title_Count":2},{"index":453,"State":"Ohio","Variety":"Nebbiolo","Avg_Points":80.0,"Avg_Price":15,"Title_Count":1},{"index":454,"State":"Ohio","Variety":"Riesling","Avg_Points":84.43,"Avg_Price":14,"Title_Count":7}]'

In [13]:
st=list(df['State'].unique())

In [14]:
st

['America',
 'Arizona',
 'California',
 'Colorado',
 'Connecticut',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Iowa',
 'Kentucky',
 'Massachusetts',
 'Michigan',
 'Missouri',
 'Nevada',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'Ohio',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'Texas',
 'Vermont',
 'Virginia',
 'Washington',
 'Washington-Oregon']

In [44]:
state_avg=pd.read_sql('prpt',engine)
stateData=state_avg[(state_avg['State']==st)]#.to_json(orient='records')

In [45]:
stateData['State'].values.tolist()

['Ohio', 'Ohio', 'Ohio', 'Ohio', 'Ohio']

In [46]:
data = [{
    "otu_ids": stateData['Avg_Points'].values.tolist(),
    "sample_values": stateData['Avg_Price'].values.tolist()
}]

In [47]:
data[0]['otu_ids']

[83.0, 84.0, 85.5, 80.0, 84.43]